In [1]:
import numpy as np
from scipy.spatial.distance import jaccard, cosine
from bs4 import BeautifulSoup
import re
import requests
import openai
import os

In [2]:
url1 = "https://www.elmundo.es/opinion/columnistas/2023/07/04/64a45a85e9cf4a65458b457f.html#ancladecomentarios"
url2 = "https://www.elmundo.es/opinion/columnistas/2023/07/04/64a45a85e9cf4a65458b457f.html"

# pad strings if needed
n1 = len(url1)
n2 = len(url2)
if n1 > n2:
    url2 = url2 + " " * abs(n1 - n2)
elif n1 < n2:
    url1 = url1 + " " * abs(n1 - n2)

len(url1), \
len(url2)

(102, 102)

Parse string to array

In [18]:
url1_arr = np.array(list(url1))
url2_arr = np.array(list(url2))
pairwise_equal = (url1_arr == url2_arr).astype(int)
pairwise_equal

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Try to shuffle second array to see how dissimilarity value rises to almost one

In [21]:
url1_arr = np.array(list(url1))
url2_arr = np.array(list(url2))
np.random.shuffle(url2_arr)
pairwise_equal = (url1_arr == url2_arr).astype(int)
print(pairwise_equal)

dissimilarity = jaccard(np.ones_like(pairwise_equal), pairwise_equal)
# The greater, the more dissimilar
dissimilarity

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


0.9509803921568627

## HTML Character entities

In [23]:
import html

entity = "&apos;"
decoded = html.unescape(entity)
print(decoded)

'


In [25]:
text = "El turista Ivan Dimitrov , de 27 años, que vive en Bristol (Reino Unido) con su novia, Hayley Bracey, y que escribió el nombre de ambos en una de las paredes del Coliseo de Roma trató de disculparse en una carta enviada a la Fiscalía de Roma, al alcalde, Roberto Gualtieri, y al ayuntamiento de la capital italiana en la que asegura que desconocía la antigüedad del monumento. &quot;Reconozco con total vergüenza que sólo después de lo sucedido me enteré de la antigüedad del monumento&quot;, escribió el joven de origen búlgaro en la carta que hoy publica el diario romano &quot;Il Messaggero&quot;. Los dos estaban visitando la capital italiana durante un viaje de tres semanas por Europa y durante una visita al Coliseo, el hombre escribió la frase &quot;Ivan+Hayley 23&quot; en una pared, como se puede ver en un vídeo colgado por otro turista en una plataforma y que se hizo viral. El ministro de Cultura italiano, Gennaro Sangiuliano, prometió entonces que se perseguiría este acto de vandalismo y después de que los Carabineros rastrearan las imágenes de las cámaras de seguridad y los registros de los hoteles y se consiguió identificar al turista mientras estaban en Bulgaria. Sangiuliano aseguró que en caso de denuncia, el ministerio de Cultura se presentará como parte civil. Se desconoce si se ha abierto algún procedimiento judicial al joven. Según el diario, el joven inmediatamente se disculpó por su gesto y dijo a los Carabineros que estaba muy arrepentido y también preocupado por las consecuencias de su acto, pues se le explicó que se arriesga a una pena de prisión de dos a cinco años y una multa de entre 2.500 y 15.000 euros. &quot;Consciente de la gravedad del acto cometido, con estas líneas quisiera dirigir mis más sinceras y sinceras disculpas a los italianos y al mundo entero por los daños causados a un bien que, de hecho, es patrimonio de toda la humanidad&quot;, escribió en la carta. Sobre todo se disculpa con el ayuntamiento de Roma que &quot;con dedicación, cuidado y sacrificio custodian el inestimable valor histórico y artístico del Coliseo&quot; y luego intenta justificarse añadiendo que &quot;sólo más tarde se enteró de la antigüedad del monumento&quot;. Esta no es la primera vez que los visitantes del enorme anfiteatro deciden vandalizar sus muros. El pasado verano una turista canadiense fue sorprendida por los agentes de seguridad del Coliseo mientras escribía su nombre con una piedra en los mismos muros. En esa ocasión, los vigilantes pudieron alertar rápidamente a la policía, que acudió al lugar y denunció a la visitante."
print(text, "\n")
decoded = html.unescape(text)
print(decoded)

El turista Ivan Dimitrov , de 27 años, que vive en Bristol (Reino Unido) con su novia, Hayley Bracey, y que escribió el nombre de ambos en una de las paredes del Coliseo de Roma trató de disculparse en una carta enviada a la Fiscalía de Roma, al alcalde, Roberto Gualtieri, y al ayuntamiento de la capital italiana en la que asegura que desconocía la antigüedad del monumento. &quot;Reconozco con total vergüenza que sólo después de lo sucedido me enteré de la antigüedad del monumento&quot;, escribió el joven de origen búlgaro en la carta que hoy publica el diario romano &quot;Il Messaggero&quot;. Los dos estaban visitando la capital italiana durante un viaje de tres semanas por Europa y durante una visita al Coliseo, el hombre escribió la frase &quot;Ivan+Hayley 23&quot; en una pared, como se puede ver en un vídeo colgado por otro turista en una plataforma y que se hizo viral. El ministro de Cultura italiano, Gennaro Sangiuliano, prometió entonces que se perseguiría este acto de vandalism

In [8]:

hiperlink = "https://elpais.com/sociedad/2023-07-03/el-gobierno-aprobara-manana-el-fin-de-las-mascarillas-obligatorias-en-centros-sanitarios-farmacias-y-residencias.html#?rel=lom"
search_spec_char = re.search("[?=&+%@#]{1}", hiperlink)
s, e = search_spec_char.span()
hiperlink[:s]

'https://elpais.com/sociedad/2023-07-03/el-gobierno-aprobara-manana-el-fin-de-las-mascarillas-obligatorias-en-centros-sanitarios-farmacias-y-residencias.html'

In [25]:
def has_http_attribute_value(tag):
    for attr in tag.attrs:
        attr_value = tag.attrs[attr]
        if isinstance(attr_value, str) and ('http://' in attr_value or 'https://' in attr_value or re.search(r"(?:\/).+(?:\/).+(?:\/?)", attr_value)) :
            return True
    return False

In [38]:
url = "https://www.elconfidencial.com/tecnologia/2023-07-04/cifrado-reglamento-europeo-abuso-menores-whatsapp-mensajeria_3684963/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=HEADERS)
response

SyntaxError: EOL while scanning string literal (2103130488.py, line 1)

In [34]:
parsed_hmtl = BeautifulSoup(response.content, "html.parser")
tags_with_urls = parsed_hmtl.find_all(lambda tag: has_http_attribute_value)
hiperlinks = []
for tags_with_url in tags_with_urls:
    for attr in tags_with_url.attrs:
        attr_value = tags_with_url.attrs[attr]
        if isinstance(attr_value, str) and ('http://' in attr_value or 'https://' in attr_value):
            hiperlink = tags_with_url[attr]
            if hiperlink.startswith("https://www.abc.es/"):
                hiperlinks.append(hiperlink)

In [35]:
hiperlinks

['https://www.abc.es/',
 'https://www.abc.es/rss/feeds/abc_portada.xml',
 'https://www.abc.es/rss/feeds/abc_ultima.xml',
 'https://www.abc.es/',
 'https://www.abc.es/',
 'https://www.abc.es/espana/elecciones-directo-pedro-sanchez-feijoo-yolanda-diaz-abascal-ultimas-noticias-generales-hoy-20230704084244-di.html',
 'https://www.abc.es/internacional/guerra-ucrania-rusia-directo-putin-zelenski-contraofensiva-ucraniana-20230705155105-di.html',
 'https://www.abc.es/espana/inmunidad-puigdemont-directo-ultima-hora-decision-justicia-ponsati-comin-hoy-20230705113122-di.html',
 'https://www.abc.es/gente/gonzalo-vargas-llosa-ultima-hora-sobre-estado-20230705121808-vi.html',
 'https://www.abc.es/play/television/noticias/audiencias-programa-ana-rosa-ganador-feijoo-sanchez-20230705120006-nt.html',
 'https://www.abc.es/recreo/entradas-taylor-swift-madrid-llegan-correos-codigo-20230705150833-nt.html',
 'https://www.abc.es/motor/dgt-confirma-situaciones-radares-pueden-multar-pese-20230705123454-nt.html'

In [64]:
url = "https://www.elconfidencial.com/espana/2022-10-12/feijoo-eliminara-ministerios-auditoria-moncloa_3505199/"
response = requests.get(url, headers=HEADERS)
parsed_hmtl = BeautifulSoup(response.content, "html.parser")
#links = [t.attrs["href"] for t in parsed_hmtl.find_all("a", href=re.compile("https?:.*"))]
p_tags = parsed_hmtl.find_all("p")
[p.text for p in p_tags]

[' El líder del PP, Alberto Núñez Feijóo, ha asegurado que las tres primeras cosas que haría de llegar al Palacio de la Moncloa sería suprimir entre seis y ocho ministerios, encargar una auditoría en las cuentas públicas y tomar en los primeros cien días "tres o cuatro decisiones estructurales". ',
 ' El líder de la oposición ha asumido estos compromisos en un desayuno informativo de El Mundo, en el que ha recalcado que el PP no renuncia a su objetivo de gobernar en solitario, algo que a su juicio lograría hacer con "pactos puntuales" si se hubiesen celebrado elecciones "anteayer". ',
 ' "Si no lo creyera no estaría tan animado como estoy. Mi partido en seis meses ha subido 10 puntos", ha señalado Feijóo al ser preguntado si prefería pactar con Vox o con el PNV. ',
 ' Por otra parte, Feijóo ha acusado al presidente del Gobierno, Pedro Sánchez, y al PSOE de "blindar al independentismo en Cataluña" y ha calificado de "sonrojante" y "humillante" que el president Pere Aragonès dijese "grac

In [61]:
search_spec_char = re.search("[=&+%@#?]{1}", "https://elpais.com/espana/2023-07-05/la-justicia-europea-retira-la-inmunidad-parlamentaria-de-carles-puigdemont.html?autoplay=1")
query_start = search_spec_char.span()[0]
hiperlink = "https://elpais.com/espana/2023-07-05/la-justicia-europea-retira-la-inmunidad-parlamentaria-de-carles-puigdemont.html?autoplay=1"[:query_start]

In [88]:
hiperlink

'https://elpais.com/espana/2023-07-05/la-justicia-europea-retira-la-inmunidad-parlamentaria-de-carles-puigdemont.html'

In [89]:
def search_articles(tag):
    return tag.name == "p" and not tag.attrs

In [92]:
url = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"

response = requests.get(url)
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

p_tags = parsed_hmtl.find_all(search_articles)
print([p.get_text() for p in p_tags])

['\n\nSuscribir\n', 'En 2016, me tocó escribir un artículo triste. Por primera vez en la historia de la humanidad, el dióxido de carbono en el aire superó las 400 partes por millón. Esa era una de las líneas rojas que tomábamos con referencia en la lucha contra el cambio climático. No era la primera que cruzábamos, pero como dice el profesor Matt England tenía un peso simbólico importante.', 'Hoy hemos cruzado otra: la temperatura media del mundo, por primera vez, ha superado los 17 grados. Eso fue el lunes 3 de julio. El 4 la temperatura subió 0,17 grados más.', "¿Líneas rojas? Sí, los esfuerzos globales por contener al cambio climático usan ciertos límites como referencia e inspiración: es una forma de medir la progresión de ese fenómeno global, sí; pero también es una forma de ponernos objetivos 'concretos'. Aunque luego no seamos capaces de conseguirlos.", 'Por eso mismo, estas líneas rojas tienen una parte importante de convención; es decir, no dejan de ser cifras elegidas con una

In [94]:
url = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"

response = requests.get(url)

In [95]:
response.ti

<Response [200]>

In [96]:
import json

In [102]:
with open("./data/extraction checkpoint.json", "r") as file:
    data = json.load(file)

In [103]:
data

{'last_media_url': 'https://www.actualarganda.com'}

In [101]:
data["last_media_url"]

'https://www.dream-alcala.com'

## Noticias a través de p tags

In [95]:
def search_p_tags_noattrs(tag):
    return (tag.name == "p" and not tag.attrs) or ("h" in tag.name and not tag.attrs)

url1 = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"
url2 = "https://okdiario.com/espana/feijoo-no-si-sera-hombre-o-mujer-pero-ya-digo-que-fiscal-general-del-estado-no-sera-del-pp-11211283"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url2, headers=headers)

### Find p or h tags, no attributes, extract text and clean new lines

In [52]:
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

p_tags = parsed_hmtl.find_all(search_p_tags_noattrs)
text_clean_from_p_tags = [re.sub("\n+", "\n", p.get_text()) for p in p_tags]
text_clean_from_p_tags[:2]

['\nAyer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer\n \n',
 '\n         Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer    \n']

In [53]:
n_ls = 1
for x in text_clean_from_p_tags:
    print(x, end="\n"*n_ls)


Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer
 


         Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer    


         Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer    


65 comentarios


Suscribir

En 2016, me tocó escribir un artículo triste. Por primera vez en la historia de la humanidad, el dióxido de carbono en el aire superó las 400 partes por millón. Esa era una de las líneas rojas que tomábamos con referencia en la lucha contra el cambio climático. No era la primera que cruzábamos, pero como dice el profesor Matt England tenía un peso simbólico importante.
Hoy hemos cruzado otra: la temperatura media del mundo, por primera vez, ha superado los 17 grados. Eso fue el lunes 3 de julio. El 4 la temperatura subió 0,17 grados más.
¿Líneas rojas? Sí, los esfuerzos globales por contener al cambio climático usan ciertos

### Extract all the text and clean new lines

In [9]:
text_clean_from_full_html = re.sub("\n+", "\n", parsed_hmtl.get_text())
text_clean_from_full_html[:50]

'\nAyer fue el día más caluroso a nivel mundial desd'

In [54]:
print(text_clean_from_full_html)


Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer
 
Webedia
Xataka
Menú
Nuevo
ANÁLISIS
XATAKA SELECCIÓN
MÓVILES
CIENCIA
GUÍAS DE COMPRA
MOVILIDAD
MAGNET
Twitch
Tiktok
Telegram
Instagram
Youtube
Twitter
PARTNERS
      Innovación LG
     
      Tecnologia Midea
     
      Hyundai Full Electric
     
      Innovación Volvo
     
      Planeta Fantasy
     
      Inspirando TCL
     
      Espacio Finetwork
     
      muévete con Renault
     
         Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer    
65 comentarios
HOY SE HABLA DE
DGT
IA
OpenAI
SkyShowtime
PlayStation VR2
Whatsapp
Inteligencia Artificial
ChatGPT
Chromecast
Ibai
Ofrecido por LG
¿Cuál es el mejor altavoz para tu próxima fiesta? Este lo tiene todo (potencia, batería, efectos…)
      Ofrecido por TCL
     
¿Qué tiene que ver la tecnología con el deporte? Para TCL mucho y estas son las razones
      Ofrecido por Minot

In [55]:
print(f"Number of characters from p tags: {len(''.join(text_clean_from_p_tags))}")
print(f"Number of characters from full text: {len(text_clean_from_full_html)}")

Number of characters from p tags: 4439
Number of characters from full text: 12608


In [56]:
text_chatgpt = """
Title: Feijóo: "El Fiscal General del Estado no será del PP"

Body:
En una entrevista con OKDIARIO, Alberto Núñez Feijóo, líder del Partido Popular y candidato a la presidencia, afirmó que el Fiscal General del Estado no será miembro del PP. Feijóo señaló que no nombrará a alguien que sea parte del Comité Ejecutivo de su partido o a alguien recién designado en otro órgano. Hizo referencia a experiencias previas en las que se criticó el nombramiento de un miembro del partido como Fiscal General o como magistrado del Tribunal Constitucional.

Feijóo también abordó otros temas en la entrevista, como la situación en el Ministerio de Justicia y la necesidad de contar con profesionales competentes en el sistema judicial. Además, mencionó su intención de recuperar los delitos de sedición y malversación en su versión anterior en los primeros 100 días de su mandato, si los españoles le otorgan el mandato.

Tags: Alberto Núñez Feijóo, Fiscal General del Estado, Partido Popular, Ministerio de Justicia, nombramientos, delito de sedición, delito de malversación.
"""

In [57]:
print(f"Number of characters from chatgpt: {len(text_chatgpt)}")

Number of characters from chatgpt: 1067


In [58]:
chars_in_token = [2, 3, 4]
for chars in chars_in_token:
    print(f"Case token is {chars} characters")
    print(f"\tNumber of tokens from p tags: {len(''.join(text_clean_from_p_tags)) // chars}")
    print(f"\tNumber of tokens from full text: {len(text_clean_from_full_html) // chars}")
    print(f"\tNumber of tokens from chatgpt: {len(text_chatgpt) // chars}", end="\n"*2)

Case token is 2 characters
	Number of tokens from p tags: 2219
	Number of tokens from full text: 6304
	Number of tokens from chatgpt: 533

Case token is 3 characters
	Number of tokens from p tags: 1479
	Number of tokens from full text: 4202
	Number of tokens from chatgpt: 355

Case token is 4 characters
	Number of tokens from p tags: 1109
	Number of tokens from full text: 3152
	Number of tokens from chatgpt: 266



### Using openai to extract keys

In [15]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [74]:
#prompt_role_complex = """Your commitment is to review text from news, which contain the headline and body content. However, there might be other pieces of text irrelevant which are irrelevant, like footer references to images related to the news, social media platforms footers, subscription texts, users' comments or texts related to comments, among others. Therefore, you have to be able to extract the content / body, the headline / title of the news and, the tags related to the news, if possible , while avoiding irrelevant text. The headline / title and content / body of the news should keep the core and details of the information contained, but you must keep a tone of independence of opinion like a journalist. Do not notify at the output about any other matters, like what you recommend or what you removed from the original text. This is the text: \n"""
#prompt_role_simple = "You receive news. Extract its title, the main content, tags (if possible), creation date (if possible) and update date (if possible). This is the news: "
with open("./data/role prompt_keys extraction.txt", "r") as file:
    prompt_role_complex = file.read()
prompt_order_full = text_clean_from_full_html
#prompt_order_p_tags = '\n'.join(text_clean_from_p_tags)
#total_prompt = prompt_role + prompt_order
#total_prompt
prompt_order_full

'\nAyer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer\n \nWebedia\nXataka\nMenú\nNuevo\nANÁLISIS\nXATAKA SELECCIÓN\nMÓVILES\nCIENCIA\nGUÍAS DE COMPRA\nMOVILIDAD\nMAGNET\nTwitch\nTiktok\nTelegram\nInstagram\nYoutube\nTwitter\nPARTNERS\n      Innovación LG\n     \n      Tecnologia Midea\n     \n      Hyundai Full Electric\n     \n      Innovación Volvo\n     \n      Planeta Fantasy\n     \n      Inspirando TCL\n     \n      Espacio Finetwork\n     \n      muévete con Renault\n     \n         Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer    \n65 comentarios\nHOY SE HABLA DE\nDGT\nIA\nOpenAI\nSkyShowtime\nPlayStation VR2\nWhatsapp\nInteligencia Artificial\nChatGPT\nChromecast\nIbai\nOfrecido por LG\n¿Cuál es el mejor altavoz para tu próxima fiesta? Este lo tiene todo (potencia, batería, efectos…)\n      Ofrecido por TCL\n     \n¿Qué tiene que ver la tecnología con el deporte? Para TCL

In [75]:
prompt_role_complex

'You are a journalist. Extract information from news articles that I will send:\nHeadline: [headline]\nTopics: [topics]\nCreation DateTime: [creation_datetime]\nUpdate DateTime: [update_datetime]\nBody Summary: [body summary]\n\nReplace [headline], [body summary], [topics], [creation_datetime], and [update_datetime] with \nthe extracted information from the provided text. The body summary instructions are: make a summary \nin the third person. Keep the original language. Do not mention the article itself. \nAdd most relevant details. Expect incoming text news.'

In [64]:
print(prompt_order_full)


Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer
 
Webedia
Xataka
Menú
Nuevo
ANÁLISIS
XATAKA SELECCIÓN
MÓVILES
CIENCIA
GUÍAS DE COMPRA
MOVILIDAD
MAGNET
Twitch
Tiktok
Telegram
Instagram
Youtube
Twitter
PARTNERS
      Innovación LG
     
      Tecnologia Midea
     
      Hyundai Full Electric
     
      Innovación Volvo
     
      Planeta Fantasy
     
      Inspirando TCL
     
      Espacio Finetwork
     
      muévete con Renault
     
         Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer    
65 comentarios
HOY SE HABLA DE
DGT
IA
OpenAI
SkyShowtime
PlayStation VR2
Whatsapp
Inteligencia Artificial
ChatGPT
Chromecast
Ibai
Ofrecido por LG
¿Cuál es el mejor altavoz para tu próxima fiesta? Este lo tiene todo (potencia, batería, efectos…)
      Ofrecido por TCL
     
¿Qué tiene que ver la tecnología con el deporte? Para TCL mucho y estas son las razones
      Ofrecido por Minot

In [102]:
openai_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_role_complex},
        {"role": "user", "content": prompt_order_full},
    ]
)
message_content = openai_response["choices"][0].message["content"]
print(message_content)

Headline: Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer
Topics: Cambio climático, récord de temperatura, calentamiento global
Creation DateTime: 2023-07-05T10:01:34Z 
Update DateTime: - 
Body Summary: Ayer se registró el día más caluroso a nivel mundial desde que se tienen registros. La temperatura media del mundo superó los 17 grados, marcando un nuevo récord. Este incremento en la temperatura muestra que el calentamiento global se está acelerando, ya que el calentamiento entre 2006 y 2020 se ha incrementado cuatro veces más rápido que en el período entre 1961 y el año 2000. Además, se señala que las emisiones de contaminantes podrían haber enfriado artificialmente ciertas áreas del océano, pero la interrupción del flujo de cargueros debido a la pandemia y las regulaciones sobre emisiones podrían haber llevado al calentamiento inesperado del océano. Estos eventos resaltan la complejidad del problema climático y la importancia de 

In [103]:
len(message_content.split("\n"))

5

In [90]:
data = [x.split(": ")[1].strip() for x in message_content.split("\n")]
keys = (("title", 0), ("body", 4), ("creation_date", 2), ("update_date", 3), ("tags", 1))
data = {key: data[idx] for (key, idx)  in keys}
data

{'title': 'Ayer fue el día más caluroso a nivel mundial desde que hay registros. El anterior récord fue anteayer',
 'body': 'Ayer se registró el día más caluroso a nivel mundial desde que se tienen registros. La temperatura media del mundo superó los 17 grados, marcando un nuevo récord. Este incremento en la temperatura muestra que el calentamiento global se está acelerando, ya que el calentamiento entre 2006 y 2020 se ha incrementado cuatro veces más rápido que en el período entre 1961 y el año 2000. Además, se señala que las emisiones de contaminantes podrían haber enfriado artificialmente ciertas áreas del océano, pero la interrupción del flujo de cargueros debido a la pandemia y las regulaciones sobre emisiones podrían haber llevado al calentamiento inesperado del océano. Estos eventos resaltan la complejidad del problema climático y la importancia de tomar medidas estratégicas para evitar problemas futuros.',
 'creation_date': '2023-07-05T10:01:34Z',
 'update_date': '-',
 'tags': 

### Only body summary

In [122]:
prompt_role_summary_only = """
You are a journalist. Output the next format:

Body Summary: [body summary]

Replace [body summary] with the summary from the incoming text body. 
The summarization instructions are: make a summary in third-person 
point of view. Translate the summary into the incoming text language. 
Do not mention the article itself. Add only important details. 
This is the text to summarize:
"""

In [123]:
body = """
España mira al cielo con cara de no entender. Junio, tradicionalmente el primero de los meses secos, ha sido el más lluvioso del año en ciudades como Málaga o Huesca. En Madrid, incluso, deja una tormenta de granizo. Pero no es suficiente. Las precipitaciones anuales continúan por debajo de los valores normales. Han sido demasiado cortas e irregulares, y los embalses siguen sin recuperarse. Estas infraestructuras han visto menguar su volumen de agua almacenada y se encuentran al 47,4% de su capacidad, una décima menos que la semana anterior. Son datos inferiores a los de las mismas fechas de 2022 y se sitúan 19 puntos por debajo del valor medio de la última década (66,7%). La situación preocupa. Por su parte, la cantidad de agua embalsada en las presas destinadas a la producción de energía eléctrica también ha menguado. Su reserva de agua se encuentra 7 décimas por debajo con respecto a la última semana, de forma que estos embalses se encuentran al 64,3% de su capacidad, igualmente por debajo de los valores medios de la última década (77%). Una cosa es clara: será preciso continuar con la pertinaz lucha que nuestro país mantiene desde toda la vida contra la sequía. En este panorama, surgen algunas preguntas: ¿nuestras reservas de agua están garantizadas? ¿Está asegurada el agua de uso humano, los caudales ecológicos o la producción de energía? ¿Qué hacen las centrales hidroeléctricas en épocas de sequía? Santiago Domínguez, responsable de generación hidroeléctrica de Endesa en España, tiene claro que este año está siendo malo. “El año pasado fue seco, por debajo de la media, y este año está siendo peor”. Arrastramos, dice, una situación de estrés hídrico que se ha ido encadenando. Para el trabajador de Endesa, que tiene centrales distribuidas por casi todo el mapa, desde Galicia a Cataluña, Extremadura o Andalucía, la situación de escasez no es nueva en el sur. Pero este año es “enorme” y ahora mismo hay una “situación de estrés hídrico” en las 3 zonas en las que operan. “Nuestras reservas se encuentran por debajo de la media nacional y están próximas al 30% con respecto a la capacidad total”, apunta, sin paliativos. “Una de las mayores, la de Mequinenza, en el Ebro, nunca ha estado en niveles tan bajos”. De hecho, tres de ellas (una en la cuenca del Ebro y dos en la cuenca del Guadiana) ya se encuentran paradas por falta de agua disponible para poder turbinar y generar electricidad. “El año pasado teníamos la previsión de que tendríamos que parar hacia septiembre-octubre, pero llegaron lluvias y no hubo que pararlas. Este año la previsión es que paren en agosto-septiembre, y en agosto es improbable que llueva…”, explica. Pero ¿por qué para una central hidroeléctrica? A grandes rasgos, por el bajo nivel de los embalses. “Cuando la entrada de agua no permite mantener la salida, la central se detiene para asegurar los caudales ecológicos, el abastecimiento humano y el riego”. En España, la ley establece una utilidad estricta para el uso del agua de los embalses: agua para consumo humano, agua para el campo y usos industriales, agua para generar energía y agua de uso recreativo. “El primero es el uso prioritario del agua, es decir, si peligra el consumo humano, todo lo que viene detrás se ajusta”, explica Domínguez. “Por eso el agua consuntiva, para el consumo humano, jamás se ha restringido (salvo excepciones muy particulares en las que la reserva es muy pequeña, no así en grandes embalses), pero sí se restringe el agua en otros sectores”, detalla. “En segunda posición se encuentra el agua para riego, cuyo consumo es muy relevante, muchísimo mayor que el humano y el hidroeléctrico, y este año ha sufrido ciertas restricciones importantes por falta de agua”, continúa. En tercer lugar se encuentra el uso hidroeléctrico, que no consume el agua. “La toma de un tramo del río, la turbina y la devuelve un poco más abajo, aunque es cierto que en ese tramo que une ambas partes el agua no está en el cauce, pero es un agua que como tal no se consume”, constata Domínguez. Para remediarlo, las Confederaciones Hidrográficas establecen una serie de medidas, como los caudales ecológicos, ya que la preservación de la biodiversidad de estas instalaciones es una obligación legal, incluso en situaciones de sequía. “Siempre se deben respetar los caudales ecológicos, por preservar el valor ecológico de la biodiversidad en nuestros ríos. Por lo que un río nunca se puede secar. El caudal mínimo debe respetarse antes del riego y después del consumo humano. El incumplimiento puede tener consecuencias penales. Si cualquier agente no lo respetase se expondría a una condena legal”, afirma. “La hidroeléctrica está en tercera posición, así que gestionamos nuestras instalaciones enfocados a asegurar los otros usos -continúa-. En los embalses donde hay reserva de agua suficiente, se desembalsa al ritmo que fijan las Confederaciones Hidrográficas para que en el río haya caudal suficiente como para atender todos los fines”. ¿Quién gestiona los embalses?Aunque pueda pensarse lo contrario, las eléctricas no deciden libremente cuándo utilizar el agua de un embalse para generar electricidad. Es la Administración del Estado quien gestiona las reservas de agua, a través de variedad de organismos regionales (Organismos de Cuenca) y, dentro de ellos, las Comisiones de Desembalse. El objetivo de estas comisiones es coordinar la variedad de necesidades de todos los beneficiarios del agua para garantizar el cumplimiento de sus usos prioritarios. “Por ejemplo, en el caso del río Ebro, hay embalses que deben garantizar el abastecimiento de Zaragoza, el riego de los campos, el agua para las industrias que la necesitan y las hidroeléctricas para generar electricidad”, ejemplifica Domínguez. En estos casos, se organizan periódicamente Comisiones de Desembalse integradas por agentes de todos los sectores. “En ellas se deben poner de acuerdo y a veces es complicado porque no hay agua para atender todos los usos. En estos casos, cada agente recibe unas instrucciones: a los regantes les dicen, por ejemplo, que pueden usar sólo una parte del volumen autorizado en condiciones normales. Y lo mismo con el resto de usos. Todos están sometidos al mejor reparto posible del volumen de agua disponible”, comenta el responsable. En la actualidad, aunque una mayor producción hidroeléctrica contribuiría a mitigar el impacto del precio del gas y la crisis climática, al reducir la necesidad de generar energía mediante centrales térmicas, nos encontramos en un año en el que la aportación hidroeléctrica está muy limitada debido a la escasez de agua. Para ello, “contamos con otro tipo de centrales, como las fotovoltaicas, que este año producirán algo más que en años anteriores por tamaño y número de instalaciones”. Asimismo, las compañías tratan de reducir la necesidad del uso del gas cubriendo la demanda con energía hidroeléctrica fuera de las horas en las que las tecnologías renovables como la solar no están disponibles. Por ejemplo, potenciando la generación hidroeléctrica por la noche. ¿Y si nos enfrentáramos a una sequía que pusiera en riesgo los usos consuntivos del agua, qué pasaría con la producción hidroeléctrica? “La primera vía sería limitar su uso al máximo, empezando por la generación hidroeléctrica. Nos ajustarían el caudal que podemos tomar para evitar situaciones de estrés en estos tramos del río entre el punto de toma y el de restitución del agua al cauce o lo restringirían del todo”, argumenta. Después se limitaría el regadío, llegando “al extremo de cancelarse del todo”. Y, ya por último, se podrían modificar los caudales ecológicos establecidos por ley. Llegados a ese punto hipotético de catástrofe futurista, explica, habría que cerrar el río para que todos los embalses pudieran sostener el agua que abastece a las ciudades. Por fortuna, “estamos muy lejos de comprometer el abastecimiento y los caudales ecológicos mínimos que aseguran la biodiversidad y el suministro. Por mucha sequía que atravesemos, estaremos siempre lejos de comprometerlo”, aclara. “Las normas y mecanismos que regulan el agua de los embalses en España son claras y, en nuestro país, somos expertos en gestión del agua y en sus problemas adicionales, tanto en sequía como en inundaciones; estamos muy preparados”.
"""

In [130]:
body.split("España mira ")

['\n',
 'al cielo con cara de no entender. Junio, tradicionalmente el primero de los meses secos, ha sido el más lluvioso del año en ciudades como Málaga o Huesca. En Madrid, incluso, deja una tormenta de granizo. Pero no es suficiente. Las precipitaciones anuales continúan por debajo de los valores normales. Han sido demasiado cortas e irregulares, y los embalses siguen sin recuperarse. Estas infraestructuras han visto menguar su volumen de agua almacenada y se encuentran al 47,4% de su capacidad, una décima menos que la semana anterior. Son datos inferiores a los de las mismas fechas de 2022 y se sitúan 19 puntos por debajo del valor medio de la última década (66,7%). La situación preocupa. Por su parte, la cantidad de agua embalsada en las presas destinadas a la producción de energía eléctrica también ha menguado. Su reserva de agua se encuentra 7 décimas por debajo con respecto a la última semana, de forma que estos embalses se encuentran al 64,3% de su capacidad, igualmente por de

In [124]:
openai_response2 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_role_summary_only},
        {"role": "user", "content": body},
    ]
)
message_content2 = openai_response2["choices"][0].message["content"]
print(message_content2)

España ha experimentado un mes de junio muy lluvioso, pero las precipitaciones anuales siguen por debajo de los valores normales. Los embalses se encuentran al 47,4% de su capacidad, por debajo de los valores medios de la última década. Esto ha llevado a preocupaciones sobre las reservas de agua y la garantía de agua para uso humano, caudales ecológicos y producción de energía. Endesa, una empresa de energía, ha señalado que este año está siendo peor que el anterior en cuanto a la escasez de agua. Algunas centrales hidroeléctricas ya se han visto obligadas a parar debido a la falta de agua disponible. La gestión del agua en España es llevada a cabo por la Administración del Estado a través de organismos regionales y comisiones de desembalse. En caso de sequía, se prioriza el consumo humano y se limita el uso del agua para riego y generación de energía. Aunque la producción hidroeléctrica está limitada este año debido a la escasez de agua, se espera que las centrales fotovoltaicas compe

In [125]:
message_content2

'España ha experimentado un mes de junio muy lluvioso, pero las precipitaciones anuales siguen por debajo de los valores normales. Los embalses se encuentran al 47,4% de su capacidad, por debajo de los valores medios de la última década. Esto ha llevado a preocupaciones sobre las reservas de agua y la garantía de agua para uso humano, caudales ecológicos y producción de energía. Endesa, una empresa de energía, ha señalado que este año está siendo peor que el anterior en cuanto a la escasez de agua. Algunas centrales hidroeléctricas ya se han visto obligadas a parar debido a la falta de agua disponible. La gestión del agua en España es llevada a cabo por la Administración del Estado a través de organismos regionales y comisiones de desembalse. En caso de sequía, se prioriza el consumo humano y se limita el uso del agua para riego y generación de energía. Aunque la producción hidroeléctrica está limitada este año debido a la escasez de agua, se espera que las centrales fotovoltaicas comp